<a href="https://colab.research.google.com/github/FugaTakata/cst1_kadai01/blob/main/cst1_kadama_kawai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [175]:
!pip install networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [176]:
import json
import networkx as nx
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph
dir = '/content/drive/MyDrive/cst1/graph/'
filename = 'diamond.json'
graph = json_graph.node_link_graph(json.load(open(dir + filename)))

In [177]:
import numpy as np

In [178]:
def calc_de(m, dists, pos):
  dedxm = 0
  dedym = 0

  for i in pos:
    if i == m:
      continue
    xi, yi = pos[i]
    xm, ym = pos[m]
    norm = np.linalg.norm(np.array((xi, yi)) - np.array((xm, ym)))
    dim = dists[i][m]

    dedxm += (xi - xm) * (1 - dim / norm)
    dedym += (yi - ym) * (1 - dim / norm)
  de = np.array([[dedxm], [dedym]])

  return de

def calc_de2(m, dists, pos):
  de2dxm2 = 0

  de2dym2 = 0

  de2dxmdym = 0

  for i in pos:
    if i == m:
      continue
    xi, yi = pos[i]
    xm, ym = pos[m]
    norm = np.linalg.norm(np.array((xi, yi)) - np.array((xm, ym)))
    dim = dists[i][m]

    de2dxm2 += 1 - (dim * (yi - ym) ** 2) / norm ** 3
    de2dym2 += 1 - (dim * (xi - xm) ** 2) / norm ** 3

    de2dxmdym += dim * (((xi - xm) * (yi - ym)) / norm ** 3)

  de2 = np.array([[de2dxm2, de2dxmdym], [de2dxmdym, de2dym2]])
  return de2


In [179]:
def kamada_kawai(graph, pos, dists):
  eps = 0.1
  graph = nx.Graph()

  while True:
    max_id = None
    max_de =  None
    max_de_norm = None
    for node_id in pos:
      de = calc_de(node_id, dists, pos)
      if max_de is None or max_de_norm < np.linalg.norm(de):
        max_id = node_id
        max_de = de
        max_de_norm = np.linalg.norm(max_de)
    
    print(max_de_norm)
    if max_de_norm < eps:
      break
    
    de2 = calc_de2(max_id, dists, pos)

    de2dxm2 = de2[0][0]
    de2dym2 = de2[1][1]
    de2dxmdym = de2[0][1]

    dedxm = max_de[0][0]
    dedym = max_de[1][0]

    dx = (de2dym2 * dedxm - de2dxmdym * dedym) / (de2dxmdym ** 2 - de2dxm2 * de2dym2)
    dy = (de2dxmdym * dedxm - de2dxm2 * dedym) / (de2dxm2 * de2dym2 - de2dxmdym ** 2)

    pos[max_id][0] += dx
    pos[max_id][1] += dy
  
  return pos


In [180]:
dists = dict(nx.all_pairs_shortest_path_length(graph))
initial_pos = nx.random_layout(graph)

In [181]:
pos = kamada_kawai(graph, initial_pos, dists)
nx.draw(graph, pos)
plt.show()
for i in pos:
  if pos[i][0] != initial_pos[i][0] or pos[i][1] != initial_pos[i][1]:
    print(pos[i], initial_pos[i])

2.523597445860276
23.8072668961316
47.61483720970601
95.22977183082708
190.459572957203
380.9191650155949
761.838328523457
1523.6766555609274
3047.3533460923213
6094.706700320802
12189.413527513212
24378.827141043777
48757.65347075763
97515.3050693372
195030.61419583697
390061.2205900093
780122.4329414498
1560244.8898499538
3120489.861084658
6240979.50597533
12481959.41889027
24963917.712830327
49927836.27795197
99855672.43356267
199711348.64331442
399422702.18187207
798845408.363744
1597690820.7274878
3195381645.454976
6390763294.909952
12781526593.819906
25563053191.63981
51126106387.27962
102252212778.55923
204504425561.11847
409008851126.23694
818017702256.4739
1636035404516.9475
3272070809037.895
6544141618079.791
13088283236163.58
26176566472331.164
52353132944666.33
104706265889336.66
209412531778677.28
418825063557358.56
837650127114721.2
1675300254229446.5
3350600508458896.5
6701201016917798.0
1.34024020338356e+16
2.6804804067671204e+16
5.360960813534241e+16
1.0721921627068483

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: overflow encountered in float_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide



nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan

KeyboardInterrupt: ignored